In [1]:
!pip install pyarrow
!pip install lightgbm==2.2.3

In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder

DATA_PATH = "../input/ashrae-energy-prediction/"
LEAKDATA_PATH = "../input/leakdata/"
OUTPUT_PATH = "../input/lgb/"

import os
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

In [3]:
#train = pd.read_pickle("train.pkl")
#test = pd.read_pickle("test.pkl")

In [4]:
train = pd.read_csv(DATA_PATH + "train.csv")
test = pd.read_csv(DATA_PATH + "test.csv")
building_df = pd.read_csv(DATA_PATH + "building_metadata.csv")
weather_train = pd.read_csv(DATA_PATH + "weather_train.csv")
weather_test = pd.read_csv(DATA_PATH + "weather_test.csv")
train = train.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
test = test.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
train = train.merge(weather_train, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how='left')
test = test.merge(weather_test, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how='left')

In [5]:
for df in [train, test]:
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["hour"] = df["timestamp"].dt.hour
    df["day"] = df["timestamp"].dt.day
    df["weekend"] = df["timestamp"].dt.weekday
    df["month"] = df["timestamp"].dt.month
    df["year"] = df["timestamp"].dt.year
    df["yearmonth"] = (2018-df["timestamp"].dt.year) * 12 + df["timestamp"].dt.month

In [6]:
le = LabelEncoder()
train["primary_use"] = le.fit_transform(train["primary_use"])
test["primary_use"] = le.fit_transform(test["primary_use"])

In [7]:
func = {'air_temperature':['mean', 'median', 'min', 'max'],
        'cloud_coverage':['mean', 'median', 'min', 'max'],
        'dew_temperature':['mean', 'median', 'min', 'max']}
g1 = train.groupby(['building_id', 'year', 'month', 'day']).agg(func)
g2 = test.groupby(['building_id', 'year', 'month', 'day']).agg(func)
g1.columns = ['_'.join(col) for col in g1.columns]
g2.columns = ['_'.join(col) for col in g2.columns]
train = train.merge(g1, on=['building_id', 'year', 'month', 'day'], how='left')
test = test.merge(g2, on=['building_id', 'year', 'month', 'day'], how='left')

In [8]:
enc = pd.concat([train[['building_id']], test[['building_id']]])['building_id'].value_counts().to_dict()
train['building_id_count'] = train['building_id'].map(enc)
test['building_id_count'] = test['building_id'].map(enc)

In [9]:
g = train.groupby(['building_id', 'meter']).agg({'meter_reading':['var', 'std', 'min', 'max']})
g.columns = ['_'.join(col) for col in g.columns]
train = train.merge(g, on=['building_id', 'meter'], how='left')
test = test.merge(g, on=['building_id', 'meter'], how='left')

In [10]:
g = train.groupby(['building_id', 'meter', 'month']).agg({'meter_reading':['std']})
g.columns = ['_'.join(col) for col in g.columns]
g = g.groupby(['building_id', 'meter']).agg({'meter_reading_std':['std']})
g.columns = ['_'.join(col) for col in g.columns]
train = train.merge(g, on=['building_id', 'meter'], how='left')
test = test.merge(g, on=['building_id', 'meter'], how='left')

In [11]:
site4 = pd.read_csv(LEAKDATA_PATH+"site4_leak_rrr_v1.csv")
site4['timestamp'] = pd.to_datetime(site4["timestamp"])
train = train.merge(site4, on=['building_id', 'timestamp'], how='left')
test = test.merge(site4, on=['building_id', 'timestamp'], how='left')

In [12]:
remove_features = ['year','timestamp','meter_reading', "precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed"]
features = []
for col in train.columns:
    if col not in remove_features:
        features.append(col)
#for col in ['building_id', 'site_id','meter','primary_use','square_feet','year_built','floor_count']:
#    try:features.remove(col)
#    except:None

In [13]:
categorical_feats = []

## Preprocessing

## Delete first rows with 0

In [14]:
train = train.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-21")')
train = train.query('not (building_id == 29 & meter == 0 & timestamp <= "2016-08-12")')
train = train.query('not (building_id == 29 & meter == 0 & timestamp <= "2016-08-12")')
train = train.query('not (building_id == 7 & meter == 1 & timestamp <= "2016-11-29" & timestamp >= "2016-11-03")')
train = train.query('not (building_id == 954 & timestamp <= "2016-08-09")')
train = train.query('not (building_id == 783 & meter == 2 & timestamp <= "2016-12-10")')
train = train.query('not (building_id == 1250 & meter == 2 & timestamp <= "2016-12-22")')
train = train.query('not (building_id == 1250 & meter == 2 & timestamp <= "2016-12-22")')
train = train.query('not (building_id == 1250 & meter == 0 & meter_reading == 0)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id <= 1280 & building_id >= 1275 & meter == 0 )')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id <= 1300 & building_id >= 1294 & meter == 0 )')
train = train.query('not (building_id == 1130 & meter == 1 & timestamp < "2016-10-01")') #start
train = train.query('not (building_id == 1303 & meter == 2 & timestamp < "2016-06-07")') #start
train = train.query('not (building_id == 1303 & meter == 0 & meter_reading < 400)') #delete nulls
train = train.query('not (building_id == 1238 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1238 & meter == 2 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1427 & meter == 2 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1425 & meter == 2 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1392 & meter == 2 & meter_reading < 100)') #delete nulls
train = train.query('not (building_id == 1391 & meter == 2 & meter_reading < 10)') #delete nulls
train = train.query('not (building_id == 1383 & meter == 2 & meter_reading < 10)') #delete nulls
train = train.query('not (building_id == 1383 & meter == 0 & meter_reading < 10)') #delete nulls
train = train.query('not (building_id == 1382 & meter == 2 & meter_reading < 10)') #delete nulls
train = train.query('not (building_id == 1323 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1322 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1321 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1320 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1317 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1301 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1288 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1287 & meter == 2 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1285 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1284 & meter == 2 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1233 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1234 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1246 & meter == 0 & meter_reading < 1)')
train = train.query('not (building_id == 1246 & meter == 1 & timestamp < "2016-03-07")')
train = train.query('not (building_id == 1246 & meter == 3 & timestamp < "2016-03-07")')
train = train.query('not (building_id == 1319 & meter == 0 & meter_reading < 1)')
train = train.query('not (building_id == 1302  & timestamp > "2016-09-28" & timestamp < "2016-10-21")')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1300 & meter == 1 )')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1300 & meter == 2 )')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1299 & meter == 2 )')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1298 & meter == 2 )')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1297)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1296)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1295)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1294)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1293)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1292)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1291)')
train = train.query('not (timestamp >= "2016-07-06 23:00:00" & timestamp <= "2016-08-12 23:00:00" & building_id == 1288 & meter == 1)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1276 & meter == 3)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1275 & meter == 3)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1272)')
train = train.query('not (timestamp >= "2016-08-21 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1264)')
train = train.query('not (timestamp >= "2016-11-09 23:00:00" & timestamp <= "2016-11-21 23:00:00" & building_id == 1263 & meter == 1)')
train = train.query('not (timestamp >= "2016-11-09 23:00:00" & timestamp <= "2016-11-21 23:00:00" & building_id == 1263 & meter == 2)')
train = train.query('not (timestamp >= "2016-08-20 23:00:00" & timestamp <= "2016-09-05 23:00:00" & building_id == 1252 & meter == 3)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1223)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1225)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1226)')
train = train.query('not (timestamp >= "2016-06-22 23:00:00" & timestamp <= "2016-08-31 23:00:00" & building_id == 1232 & (meter == 1 or meter == 3))')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1233 )')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1234 )')
train = train.query('not (meter == 3 & timestamp >= "2016-04-03 23:00:00" & timestamp <= "2016-05-09 23:00:00" & building_id == 1235 )')
train = train.query('not (timestamp >= "2016-04-20 23:00:00" & timestamp <= "2016-05-02 23:00:00" & building_id == 1240 )')
train = train.query('not (meter == 3 & timestamp >= "2016-04-03 23:00:00" & timestamp <= "2016-05-06 23:00:00" & building_id == 1244 )')
train = train.query('not (meter == 2 & timestamp >= "2016-10-18 23:00:00" & timestamp <= "2016-11-08 23:00:00" & building_id == 1243 )')
train = train.query('not (meter == 0 & timestamp >= "2016-07-13 00:00:00" & timestamp <= "2016-11-20 00:00:00" & building_id == 1241 )')
train = train.query('not (meter == 0 & timestamp >= "2016-09-14 00:00:00" & timestamp <= "2016-09-30 00:00:00" & building_id == 1232 )')
train = train.query('not (meter == 3 & timestamp <= "2016-02-26 00:00:00" & building_id == 1227 )')
train = train.query('not (meter == 2 & timestamp <= "2016-02-15 00:00:00" & building_id == 1189 )')
train = train.query('not (meter == 2 & timestamp <= "2016-02-11 00:00:00" & building_id == 1176 )')
train = train.query('not (meter == 1 & timestamp >= "2016-09-01 00:00:00" & timestamp <= "2016-09-22 00:00:00" & building_id == 1226 )')
train = train.query('not (meter == 1 & timestamp >= "2016-09-01 00:00:00" & timestamp <= "2016-09-22 00:00:00" & building_id == 1225 )')
train = train.query('not (meter == 0 & timestamp <= "2016-08-04 00:00:00"& building_id == 1169 )')
train = train.query('not (meter == 0 & timestamp <= "2016-01-21 00:00:00"& building_id == 1153 )')
train = train.query('not (meter == 0 & timestamp <= "2016-09-08 00:00:00"& building_id == 1094 )')
train = train.query('not (meter == 0 & timestamp <= "2016-10-25 14:00:00"& building_id == 1073 )')
train = train.query('not (meter == 0 & timestamp >= "2016-07-13 00:00:00" & timestamp <= "2016-11-20 00:00:00" & building_id == 1241 )')
train = train.query('not (meter == 0 & timestamp >= "2016-01-03 00:00:00" & timestamp <= "2016-03-09 00:00:00" & building_id == 1234 )')
train = train.query('not (meter == 0 & timestamp >= "2016-01-03 00:00:00" & timestamp <= "2016-03-09 00:00:00" & building_id == 1233 )')
train = train.query('not (meter == 0 & timestamp >= "2016-09-14 00:00:00" & timestamp <= "2016-10-01 00:00:00" & building_id == 1232 )')
train = train.query('not (meter == 3 &  timestamp <= "2016-02-25 00:00:00" & building_id == 1227 )')
train = train.query('not (meter == 1 & timestamp >= "2016-08-25 00:00:00" & timestamp <= "2016-10-01 00:00:00" & building_id == 1226 )')
train = train.query('not (meter == 1 & timestamp >= "2016-08-25 00:00:00" & timestamp <= "2016-10-01 00:00:00" & building_id == 1225 )')
train = train.query('not (timestamp >= "2016-07-01 15:00:00" & timestamp <= "2016-07-05 04:00:00" & site_id == 9)')

In [15]:
building_ids = [1250, 1331, 60, 1238, 1072, 783]
meters = [2, 2, 1, 2, 2, 2]
months = [12, 9, 5, 6, 8, 12]

for building_id, month, meter in zip(building_ids, months, meters):
    train = train[((train['building_id'] == building_id) & (train['month'] < month) & (train['meter'] <= meter)) == False]

In [16]:
building_ids = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,30,32,34,36,38,40,42,44,45,47,49,50,51,52,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,106,112,190,259,455,681,761,777,799,802,895,604,48,29,1221,1169,1113,1094,1073,997,954,904,848,559,43,41,39,37,35,33,31,783,1331,1303,1290,1246,1246,1238,1160,1130,1072,1072,1003,954,954,236]
meters =       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,1,3,1,2,1,1,2,1,1,2,1,1]
months =       [6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,7,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,11,11,2,4,6,5,9,5,11,9,5,11,5,8,10,7,7,8,10,6,8,2,4,3,5,5,5,5,5,5,5,11,9,6,2,2,2,5,10,9,7,3,2,8,8,3]

for building_id, month, meter in zip(building_ids, months, meters):
    train = train[((train['building_id'] == building_id) & (train['month'] < month) & (train['meter'] <= meter)) == False]

In [17]:
train = train.query('not (meter == 0 & meter_reading == 0 )')

In [18]:
train.to_pickle(OUTPUT_PATH + "train.pkl")
test.to_pickle(OUTPUT_PATH + "test.pkl")

## Outlier detection

In [19]:
#import heapq
#for building_id in train['building_id'].unique():
#    for meter in train[train['building_id'] == building_id]['meter'].unique():
#        g = train[train['building_id'] == building_id][train['meter'] == meter]
#        largest = heapq.nlargest(5, g['meter_reading'])
#        if len(largest) > 2:
#            if largest[0] > largest[1] * 2:
#                #print(building_id, meter, largest[0], len(train))
#                train.loc[((train.building_id == building_id) & (train.meter_reading == largest[0]) & (train.meter == meter)), 'meter_reading'] = np.nan
#                #print(len(train))

In [20]:
def get_clf(site_id):
    if site_id in [0,1,2,3,4,15]:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.2,feature_fraction=0.9,subsample=0.2,  subsample_freq=1,num_leaves=20, metric='rmse')
    elif site_id == 5:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.17279828575424924,feature_fraction= 0.5014962543722731,max_depth=11,subsample=0.9679640078506427,num_leaves=10, metric='rmse')
    elif site_id == 6:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.08080772525947343,feature_fraction= 0.622900028560716,max_depth=13,subsample=0.10067186032099355,num_leaves=34, metric='rmse')
    elif site_id == 7:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.18299773578701567,feature_fraction= 0.6223879954919564,max_depth=10,subsample=0.9414223121432328,num_leaves=10, metric='rmse')
    elif site_id == 8:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.09189872129725521,feature_fraction= 0.9753756863983394,max_depth=10,subsample=0.9832629640612986,num_leaves=11, metric='rmse')
    elif site_id == 9:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.08255250790364155,feature_fraction= 0.5369661052821386,max_depth=16,subsample=0.9833827601905479,num_leaves=35, metric='rmse')
    elif site_id == 10:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.2,feature_fraction= 0.5000000000000000,max_depth=8,subsample=1,num_leaves=21, metric='rmse')
    elif site_id == 11:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.20000000000000000,feature_fraction= 0.5000000091689872,max_depth=10,subsample=0.6941231045450234,num_leaves=24, metric='rmse')
    elif site_id == 12:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.17443329555575438,feature_fraction= 0.9734466427044206,max_depth=16,subsample=0.1137291435238815,num_leaves=10, metric='rmse')
    elif site_id == 13:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.08338681267136139,feature_fraction= 0.5096109648903343,max_depth=15,subsample=0.9847653065775817,num_leaves=40, metric='rmse')
    elif site_id == 14:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.08085570180603563,feature_fraction= 0.506472390324229,max_depth=10,subsample=0.702980869161743,num_leaves=21, metric='rmse')

## Train

In [21]:
num_folds = 3
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for site_id in train['site_id'].unique():
    a += 1
    print(a)
    for meter in train[train['site_id'] == site_id]['meter'].unique():
        train_small = train[(train['site_id'] == site_id) & (train['meter'] == meter)]
        test_small = test[(test['site_id'] == site_id) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

            clf = get_clf(site_id)
            clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=150,verbose=2000)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            %time predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_site_id'] = predictions / num_folds
        test_small['meter_reading_site_id'] = test_small['meter_reading_site_id'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_site_id']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_site_id']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
#0.8852894403996184

1
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[59]	valid_0's rmse: 0.305582
CPU times: user 9.36 s, sys: 1.04 s, total: 10.4 s
Wall time: 820 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[113]	valid_0's rmse: 0.459239
CPU times: user 11.9 s, sys: 955 ms, total: 12.9 s
Wall time: 982 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[185]	valid_0's rmse: 0.658823
CPU times: user 15.1 s, sys: 1.02 s, total: 16.1 s
Wall time: 1.19 s
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[36]	valid_0's rmse: 1.5191
CPU times: user 1.44 s, sys: 14 ms, total: 1.45 s
Wall time: 90.7 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[43]	valid_0's rmse: 1.76775
CPU times: user 1.54 s, sys: 0 ns, total: 1.54 s
Wall time: 95.7 ms
Training until 

Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[10]	valid_0's rmse: 2.09491
CPU times: user 359 ms, sys: 23.6 ms, total: 383 ms
Wall time: 24 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[38]	valid_0's rmse: 2.30459
CPU times: user 440 ms, sys: 7.81 ms, total: 447 ms
Wall time: 28 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[14]	valid_0's rmse: 2.35655
CPU times: user 381 ms, sys: 9.01 ms, total: 390 ms
Wall time: 24.4 ms
8
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[353]	valid_0's rmse: 0.475392
CPU times: user 26.4 s, sys: 1.29 s, total: 27.7 s
Wall time: 2.04 s
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[41]	valid_0's rmse: 0.55605
CPU times: user 9.15 s, sys: 1.29 s, total: 10.4 s
Wall time: 947 ms
Training until

Early stopping, best iteration is:
[100]	valid_0's rmse: 1.11502
CPU times: user 17.5 s, sys: 1.43 s, total: 18.9 s
Wall time: 1.53 s
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[85]	valid_0's rmse: 1.26495
CPU times: user 17.3 s, sys: 1.63 s, total: 18.9 s
Wall time: 1.59 s
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[424]	valid_0's rmse: 1.73415
CPU times: user 49.2 s, sys: 1.66 s, total: 50.8 s
Wall time: 3.63 s
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[127]	valid_0's rmse: 1.32094
CPU times: user 21 s, sys: 1.54 s, total: 22.6 s
Wall time: 1.85 s
14
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[1073]	valid_0's rmse: 0.261901
CPU times: user 2min 19s, sys: 1.74 s, total: 2min 21s
Wall time: 9.59 s
Training until validation scores don't improve for 150 rounds.
Early sto

In [22]:
num_folds = 4
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for building_id in train['building_id'].unique():
    a += 1
    print(a)
    for meter in train[train['building_id'] == building_id]['meter'].unique():
        train_small = train[(train['building_id'] == building_id) & (train['meter'] == meter)]
        test_small = test[(test['building_id'] == building_id) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

            clf = lgb.LGBMRegressor(n_estimators=6000,
                                        learning_rate=0.2,
                                        feature_fraction=0.9,
                                        subsample=0.2,  
                                        subsample_freq=1,
                                        num_leaves=20,
                                        metric='rmse')
            clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=150,verbose=2000)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            %time predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_building_id'] = predictions / num_folds
        test_small['meter_reading_building_id'] = test_small['meter_reading_building_id'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_building_id']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_building_id']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))

1
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[49]	valid_0's rmse: 0.245753
CPU times: user 248 ms, sys: 0 ns, total: 248 ms
Wall time: 17.9 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[12]	valid_0's rmse: 0.164335
CPU times: user 341 ms, sys: 982 µs, total: 342 ms
Wall time: 20.6 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[7]	valid_0's rmse: 0.1538
CPU times: user 219 ms, sys: 16.5 ms, total: 235 ms
Wall time: 14.7 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[38]	valid_0's rmse: 0.256502
CPU times: user 260 ms, sys: 15.4 ms, total: 275 ms
Wall time: 17.2 ms
2
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[16]	valid_0's rmse: 0.994749
CPU times: user 242 ms, sys: 2.81 ms, total: 245 ms
Wall time: 15.3 ms
Training

Early stopping, best iteration is:
[135]	valid_0's rmse: 0.148565
CPU times: user 353 ms, sys: 9.49 ms, total: 362 ms
Wall time: 22.7 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[85]	valid_0's rmse: 0.222869
CPU times: user 303 ms, sys: 5.5 ms, total: 308 ms
Wall time: 19.3 ms
9
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[14]	valid_0's rmse: 0.0738623
CPU times: user 238 ms, sys: 6.81 ms, total: 245 ms
Wall time: 15.3 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[17]	valid_0's rmse: 0.0560757
CPU times: user 240 ms, sys: 7.09 ms, total: 247 ms
Wall time: 15.5 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[10]	valid_0's rmse: 0.0530042
CPU times: user 239 ms, sys: 7.4 ms, total: 246 ms
Wall time: 15.5 ms
Training until validation scores don't improve for 150 rounds.
E

Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[14]	valid_0's rmse: 0.88282
CPU times: user 254 ms, sys: 0 ns, total: 254 ms
Wall time: 14.4 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[1]	valid_0's rmse: 1.0405
CPU times: user 228 ms, sys: 0 ns, total: 228 ms
Wall time: 14.2 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[82]	valid_0's rmse: 2.12866
CPU times: user 320 ms, sys: 8.85 ms, total: 329 ms
Wall time: 20.6 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[9]	valid_0's rmse: 1.46319
CPU times: user 231 ms, sys: 9.58 ms, total: 240 ms
Wall time: 15.2 ms
16
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[49]	valid_0's rmse: 0.137065
CPU times: user 285 ms, sys: 0 ns, total: 285 ms
Wall time: 17.8 ms
Training until val

Early stopping, best iteration is:
[12]	valid_0's rmse: 0.14887
CPU times: user 213 ms, sys: 24.9 ms, total: 238 ms
Wall time: 14.9 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[16]	valid_0's rmse: 0.246838
CPU times: user 242 ms, sys: 3.36 ms, total: 246 ms
Wall time: 15.5 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.257383
CPU times: user 252 ms, sys: 8.46 ms, total: 260 ms
Wall time: 16.3 ms
25
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[36]	valid_0's rmse: 0.191877
CPU times: user 257 ms, sys: 4.6 ms, total: 262 ms
Wall time: 16.4 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[3]	valid_0's rmse: 0.176323
CPU times: user 194 ms, sys: 122 µs, total: 194 ms
Wall time: 14.3 ms
Training until validation scores don't improve for 150 rounds.
Early s

Early stopping, best iteration is:
[185]	valid_0's rmse: 0.266506
CPU times: user 491 ms, sys: 15.4 ms, total: 506 ms
Wall time: 31.7 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[6]	valid_0's rmse: 1.45298
CPU times: user 211 ms, sys: 12.8 ms, total: 224 ms
Wall time: 14 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[2]	valid_0's rmse: 1.66359
CPU times: user 207 ms, sys: 14.9 ms, total: 221 ms
Wall time: 13.8 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[5]	valid_0's rmse: 1.396
CPU times: user 217 ms, sys: 8.73 ms, total: 226 ms
Wall time: 14.1 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[1]	valid_0's rmse: 2.02409
CPU times: user 207 ms, sys: 10.7 ms, total: 217 ms
Wall time: 13.6 ms
35
Training until validation scores don't improve for 150 rounds.
Early stoppin

...

Early stopping, best iteration is:
[16]	valid_0's rmse: 0.182603
CPU times: user 219 ms, sys: 17.7 ms, total: 237 ms
Wall time: 14.8 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[23]	valid_0's rmse: 0.288334
CPU times: user 257 ms, sys: 4.39 ms, total: 262 ms
Wall time: 16.4 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[114]	valid_0's rmse: 0.275836
CPU times: user 363 ms, sys: 20.2 ms, total: 383 ms
Wall time: 23.9 ms
1415
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[10]	valid_0's rmse: 0.197495
CPU times: user 254 ms, sys: 98 µs, total: 254 ms
Wall time: 14.2 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[23]	valid_0's rmse: 0.203849
CPU times: user 255 ms, sys: 132 µs, total: 255 ms
Wall time: 15.1 ms
Training until validation scores don't improve for 150 rounds.
Ea

Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[25]	valid_0's rmse: 0.304431
CPU times: user 235 ms, sys: 10.4 ms, total: 245 ms
Wall time: 15.3 ms
1425
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[16]	valid_0's rmse: 0.200717
CPU times: user 222 ms, sys: 9.52 ms, total: 232 ms
Wall time: 14.5 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[11]	valid_0's rmse: 0.174972
CPU times: user 218 ms, sys: 6.62 ms, total: 225 ms
Wall time: 14.1 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[7]	valid_0's rmse: 0.121793
CPU times: user 219 ms, sys: 2.48 ms, total: 221 ms
Wall time: 13.8 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[36]	valid_0's rmse: 0.174422
CPU times: user 271 ms, sys: 23 ms, total: 294 ms
Wall time: 18.4 ms
142

Early stopping, best iteration is:
[15]	valid_0's rmse: 1.60469
CPU times: user 254 ms, sys: 0 ns, total: 254 ms
Wall time: 14.2 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[10]	valid_0's rmse: 0.947152
CPU times: user 262 ms, sys: 4.5 ms, total: 266 ms
Wall time: 13.8 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[12]	valid_0's rmse: 0.647203
CPU times: user 224 ms, sys: 7.2 ms, total: 231 ms
Wall time: 14.5 ms
1432
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[18]	valid_0's rmse: 0.135433
CPU times: user 231 ms, sys: 3.37 ms, total: 235 ms
Wall time: 14.7 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[19]	valid_0's rmse: 0.148953
CPU times: user 271 ms, sys: 25.4 ms, total: 296 ms
Wall time: 18.5 ms
Training until validation scores don't improve for 150 rounds.
Early 

Early stopping, best iteration is:
[68]	valid_0's rmse: 0.233674
CPU times: user 288 ms, sys: 9.47 ms, total: 298 ms
Wall time: 18.6 ms
1439
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[35]	valid_0's rmse: 0.0229151
CPU times: user 256 ms, sys: 0 ns, total: 256 ms
Wall time: 15.9 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[23]	valid_0's rmse: 0.0292478
CPU times: user 243 ms, sys: 3.49 ms, total: 247 ms
Wall time: 15.4 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[28]	valid_0's rmse: 0.0546114
CPU times: user 250 ms, sys: 3.74 ms, total: 253 ms
Wall time: 15.8 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[234]	valid_0's rmse: 0.0656344
CPU times: user 589 ms, sys: 21.6 ms, total: 610 ms
Wall time: 38.2 ms
1440
Training until validation scores don't improve for 150 r

CPU times: user 347 ms, sys: 5.32 ms, total: 353 ms
Wall time: 22.2 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[138]	valid_0's rmse: 0.0266857
CPU times: user 294 ms, sys: 3.14 ms, total: 297 ms
Wall time: 18.6 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[67]	valid_0's rmse: 0.0379409
CPU times: user 242 ms, sys: 12.1 ms, total: 254 ms
Wall time: 15.9 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[2]	valid_0's rmse: 0.15828
CPU times: user 205 ms, sys: 14.4 ms, total: 220 ms
Wall time: 13.8 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[20]	valid_0's rmse: 0.0980423
CPU times: user 220 ms, sys: 5.26 ms, total: 225 ms
Wall time: 14.1 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[48]	valid_0's rmse: 0.103148
CPU

In [23]:
num_folds = 4
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for primary_use in train['primary_use'].unique():
    a += 1
    print(a)
    for meter in train[train['primary_use'] == primary_use]['meter'].unique():
        train_small = train[(train['primary_use'] == primary_use) & (train['meter'] == meter)]
        test_small = test[(test['primary_use'] == primary_use) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

            clf = lgb.LGBMRegressor(n_estimators=6000,
                                        learning_rate=0.2,
                                        feature_fraction=0.9,
                                        subsample=0.2,  
                                        subsample_freq=1,
                                        num_leaves=20,
                                        metric='rmse')
            clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=150,verbose=2000)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            %time predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_primary_use'] = predictions / num_folds
        test_small['meter_reading_primary_use'] = test_small['meter_reading_primary_use'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_primary_use']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_primary_use']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))

1
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[296]	valid_0's rmse: 0.371892
CPU times: user 2min 48s, sys: 8.23 s, total: 2min 56s
Wall time: 15.5 s
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[1099]	valid_0's rmse: 0.30198
CPU times: user 12min 52s, sys: 8.46 s, total: 13min
Wall time: 53.9 s
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[700]	valid_0's rmse: 0.321264
CPU times: user 6min 25s, sys: 8.52 s, total: 6min 33s
Wall time: 29.3 s
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[412]	valid_0's rmse: 0.411792
CPU times: user 3min 35s, sys: 8.28 s, total: 3min 43s
Wall time: 18.8 s
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[141]	valid_0's rmse: 1.21868
CPU times: user 14.8 s, sys: 1.22 s, total: 16 s
Wall time: 1.3 

CPU times: user 3.01 s, sys: 345 ms, total: 3.36 s
Wall time: 210 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[31]	valid_0's rmse: 2.25151
CPU times: user 2.9 s, sys: 364 ms, total: 3.26 s
Wall time: 204 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[8]	valid_0's rmse: 2.22473
CPU times: user 3.04 s, sys: 440 ms, total: 3.48 s
Wall time: 219 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[43]	valid_0's rmse: 1.22815
CPU times: user 4.99 s, sys: 667 ms, total: 5.65 s
Wall time: 405 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[20]	valid_0's rmse: 1.23954
CPU times: user 4.49 s, sys: 668 ms, total: 5.16 s
Wall time: 380 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[14]	valid_0's rmse: 1.29846
CPU times: user 5.06 s

Early stopping, best iteration is:
[166]	valid_0's rmse: 0.40194
CPU times: user 4.28 s, sys: 204 ms, total: 4.48 s
Wall time: 280 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[24]	valid_0's rmse: 0.810654
CPU times: user 243 ms, sys: 2.33 ms, total: 246 ms
Wall time: 15.4 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[15]	valid_0's rmse: 0.790717
CPU times: user 284 ms, sys: 9.58 ms, total: 293 ms
Wall time: 18.3 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[58]	valid_0's rmse: 0.597579
CPU times: user 336 ms, sys: 4.84 ms, total: 341 ms
Wall time: 21.3 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[10]	valid_0's rmse: 1.6747
CPU times: user 270 ms, sys: 1.54 ms, total: 271 ms
Wall time: 17.1 ms
7
Training until validation scores don't improve for 150 rounds.
Early s

Early stopping, best iteration is:
[11]	valid_0's rmse: 0.725522
CPU times: user 559 ms, sys: 166 µs, total: 559 ms
Wall time: 35.2 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[12]	valid_0's rmse: 0.777903
CPU times: user 294 ms, sys: 11.5 ms, total: 306 ms
Wall time: 19.1 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[71]	valid_0's rmse: 0.828861
CPU times: user 576 ms, sys: 12 ms, total: 588 ms
Wall time: 36.8 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[7]	valid_0's rmse: 0.183416
CPU times: user 218 ms, sys: 13.8 ms, total: 232 ms
Wall time: 14.5 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[4]	valid_0's rmse: 0.536091
CPU times: user 217 ms, sys: 11.1 ms, total: 228 ms
Wall time: 14.2 ms
Training until validation scores don't improve for 150 rounds.
Early stop

Early stopping, best iteration is:
[13]	valid_0's rmse: 1.57182
CPU times: user 388 ms, sys: 12.6 ms, total: 401 ms
Wall time: 25.1 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[3]	valid_0's rmse: 2.16732
CPU times: user 216 ms, sys: 9.56 ms, total: 225 ms
Wall time: 14.1 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[15]	valid_0's rmse: 0.824352
CPU times: user 290 ms, sys: 7.17 ms, total: 297 ms
Wall time: 18.6 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[24]	valid_0's rmse: 1.04552
CPU times: user 231 ms, sys: 20 ms, total: 251 ms
Wall time: 15.7 ms
Training until validation scores don't improve for 150 rounds.
Early stopping, best iteration is:
[6]	valid_0's rmse: 1.39063
CPU times: user 194 ms, sys: 358 µs, total: 194 ms
Wall time: 14 ms
14
Training until validation scores don't improve for 150 rounds.
Early stoppin

In [24]:
sub = pd.read_csv(DATA_PATH + "sample_submission.csv")
#sub["meter_reading"] = (test['meter_reading_site_id'] * 0.6 + test['meter_reading_primary_use'] * 0.2 + test['meter_reading_building_id'] * 0.2)
#sub.to_csv("best_more_deleting.gz",  index = False, compression= 'gzip')

In [25]:
test["meter_reading"] = test['meter_reading_site_id'] * 0.6 + test['meter_reading_primary_use'] * 0.2 + test['meter_reading_building_id'] * 0.2

In [26]:
###Postprocess 
test['meter_reading'] = [meter_r * 0.85 if site not in [0,1,2,3,4,15] else meter_r for meter_r,site in zip(test['meter_reading'], test['site_id'])]

In [27]:
leak_df = pd.read_csv(LEAKDATA_PATH+'leak.csv')
leak_df['timestamp'] = pd.to_datetime(leak_df["timestamp"])
leak_df.dropna(0, inplace=True)
leak_df = leak_df[(leak_df.timestamp.dt.year > 2016) & (leak_df.timestamp.dt.year < 2019)]
leak_df.loc[leak_df.meter_reading < 0, 'meter_reading'] = 0 # remove large negative values
leak_df = leak_df[leak_df.building_id!=245]
leak_df['meter_reading_sc'] = leak_df['meter_reading'] 
del leak_df['meter_reading'] 
test = test.merge(leak_df, on=['building_id','meter','timestamp'], how='left')

In [28]:
#test[['row_id', 'meter_reading_all','meter_reading_site_id','meter_reading_building_id','meter_reading_primary_use','meter_reading_sc']].to_pickle('test_pred09122019.pkl')

In [29]:
test['meter_reading'] = [a if (a >= 0 and site != 4) or (a < 0 and site != 4) else b for a,b,site in zip(test['meter_reading_sc'], test['meter_reading'], test['site_id'])]

In [30]:
sub["meter_reading"] = test['meter_reading']
sub.to_csv("../sub/lgb_3wise.csv",  index = False,float_format="%.4f")